In [28]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 208def28-6474-4371-afbf-14c33cf5992d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 208def28-6474-4371-afbf-14c33cf5992d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 208def28-6474-4371-afbf-14c33cf5992d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 208def28-6474-4371-afbf-14c33cf5992d.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [34]:
from pyspark.sql.functions import explode, col, to_date
from datetime import datetime
from awsglue.dynamicframe import DynamicFrame

In [3]:
s3_path = "s3://spotify-etl-project-manas/raw_data/to_processed/"
source_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type="s3",
    connection_options={"paths": [s3_path]},
    format="json",
)

In [4]:
spotify_df = source_dyf.toDF()

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [26]:
def process_album(df): 
    df = df.withColumn("items", explode("items")).select(col("items.track.album.id").alias("album_id"), 
                                               col("items.track.album.name").alias("album_name"),
                                               col("items.track.album.release_date").alias("release_date"),
                                               col("items.track.album.total_tracks").alias("total_tracks"),
                                               col("items.track.album.external_urls.spotify").alias("url")
                                               ).drop_duplicates(['album_id'])
    return df

def process_artists(df): 
    df_items_exploded = df.select(explode(col("items")).alias("item"))
    df_artist_exploded = df_items_exploded.select(explode(col("item.track.artists")).alias("artists"))
    df_artist = df_artist_exploded.select(col("artists.id").alias("artist_id"),
                         col("artists.name").alias("artist_name"),
                         col("artists.external_urls.spotify").alias("external_url")).drop_duplicates(['artist_id'])
    return df_artist

def process_songs(df) : 
    df_items_exploded = df.select(explode(col("items")).alias("item"))
    df_songs = df_items_exploded.select(
    col("item.track.id").alias("song_id"),
    col("item.track.name").alias("song_name"),
    col("item.track.duration_ms").alias("duration_ms"),
    col("item.track.external_urls.spotify").alias("url"),
    col("item.track.popularity").alias("popularity"),
    col("item.added_at").alias("song_added"),
    col("item.track.album.id").alias("album_id"),
    col("item.track.artists")[0]["id"].alias("artist_id")
    ).drop_duplicates(['song_id'])
    
    df_songs = df_songs.withColumn("song_added",to_date(col("song_added")))
    return df_songs

In [21]:
album_df = process_album(spotify_df)
album_df.show(5)

+--------------------+--------------------+------------+------------+--------------------+
|            album_id|          album_name|release_date|total_tracks|                 url|
+--------------------+--------------------+------------+------------+--------------------+
|0DLvFVIfwt3OHdK9k...|Where I've Been, ...|  2024-05-31|          12|https://open.spot...|
|0EiI8ylL0FmWWpgHV...|The Rise and Fall...|  2023-09-22|          14|https://open.spot...|
|0VRb8B2ls0GDEfmKi...|             Soltera|  2024-09-25|           1|https://open.spot...|
|0XA403JTounqFh2ow...|          Diet Pepsi|  2024-08-09|           1|https://open.spot...|
|0mV9Pfr1GfoZLkp1E...|          Embrace It|  2024-08-23|           1|https://open.spot...|
+--------------------+--------------------+------------+------------+--------------------+
only showing top 5 rows


In [22]:

artist_df = process_artists(spotify_df)
artist_df.show(5)

+--------------------+------------+--------------------+
|           artist_id| artist_name|        external_url|
+--------------------+------------+--------------------+
|06HL4z0CvFAxyc27G...|Taylor Swift|https://open.spot...|
|0EmeFodog0BfCgMzA...|     Shakira|https://open.spot...|
|0PCCGZ0wGLizHt2KZ...|     Artemas|https://open.spot...|
|0Y5tJX1MQlPlqiwlO...|Travis Scott|https://open.spot...|
|0du5cEVh5yTK9QJze...|  Bruno Mars|https://open.spot...|
+--------------------+------------+--------------------+
only showing top 5 rows


In [27]:
song_df = process_songs(spotify_df)
song_df.show(10)

+--------------------+--------------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|             song_id|           song_name|duration_ms|                 url|popularity|song_added|            album_id|           artist_id|
+--------------------+--------------------+-----------+--------------------+----------+----------+--------------------+--------------------+
|0IsIY8pfu1yaGkPUD...|Guess featuring b...|     143330|https://open.spot...|        74|2024-10-18|36P07bti6xD99o7S1...|25uiPmTg16RbhZWAq...|
|0Sr7ssScx54yxdM2o...|Q U E V A S H A C...|     224022|https://open.spot...|        88|2024-10-18|3C5uwdRE5QRoXSGPP...|3E12tRURRvPfHz0hA...|
|0WbMK4wrZ1wFSty9F...|    Good Luck, Babe!|     218423|https://open.spot...|        95|2024-10-18|1WAjjRMfZjEXtB0lQ...|7GlBOeep6PqTfFi59...|
|0io16MKpbeDIdYzmG...|          Embrace It|     104418|https://open.spot...|        88|2024-10-18|0mV9Pfr1GfoZLkp1E...|5HK6QtizXJzCmoYTk...|
|17phhZDn6oGt

In [35]:
def write_to_s3(df, path_suffix, format_type = "csv"):
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    glueContext.write_dynamic_frame.from_options(
        frame=dynamic_frame,
        connection_type="s3",
        connection_options={"path": "s3://spotify-etl-project-manas/transformed_data/" + path_suffix},
        format=format_type
    )

In [39]:
write_to_s3(album_df,"album_data/album_tansforment_{}".format(datetime.now().strftime("%Y%m%d")),'csv')

In [40]:
write_to_s3(artist_df, "artist_data/artist_transformed_{}".format(datetime.now().strftime("%Y%m%d")), 'csv')

In [41]:
write_to_s3(song_df, "songs_data/song_transformed_{}".format(datetime.now().strftime("%Y%m%d")), 'csv')